In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def eval_metrics(y_val, predictions, probs, ave = 'micro'):
    from sklearn import metrics
    
    f1 = round(metrics.f1_score(y_val, predictions,average=ave), 3)
    precision = round(metrics.precision_score(y_val, predictions,average=ave), 3)
    recal = round(metrics.recall_score(y_val, predictions,average=ave), 3)
    roc_auc = round(metrics.roc_auc_score(y_val, probs, multi_class='ovo'), 3)
  
    return f1, precision, recal, roc_auc

# Load Data

In [ ]:
df_test = pd.read_csv('702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
df_train_val = pd.read_csv('4910797b-ee55-40a7-8668-10efd5c1b960.csv')
df_train_lab = pd.read_csv('0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')

In [ ]:
df_test.shape

(14850, 40)

In [ ]:
df_train_val.shape

(37224, 40)

In [ ]:
df_train_lab.shape

(59400, 2)

In [ ]:
df_train_val.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [ ]:
df_train_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37224 entries, 0 to 37223
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     37224 non-null  int64  
 1   amount_tsh             37224 non-null  float64
 2   date_recorded          37224 non-null  object 
 3   funder                 34982 non-null  object 
 4   gps_height             37224 non-null  int64  
 5   installer              34964 non-null  object 
 6   longitude              37224 non-null  float64
 7   latitude               37224 non-null  float64
 8   wpt_name               37224 non-null  object 
 9   num_private            37224 non-null  int64  
 10  basin                  37224 non-null  object 
 11  subvillage             36993 non-null  object 
 12  region                 37224 non-null  object 
 13  region_code            37224 non-null  int64  
 14  district_code          37224 non-null  int64  
 15  lg

In [ ]:
df_train_val.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,37224.000000,37224.000000,37224.000000,37224.000000,3.722400e+04,37224.000000,37224.000000,37224.000000,37224.000000,37223.000000
mean,37066.944122,312.066935,666.116565,34.098386,-5.702236e+00,0.446191,15.337820,5.625215,180.843407,1297.698600
std,21544.968246,3174.077810,692.087430,6.534649,2.944953e+00,12.100500,17.714636,9.619470,489.746674,952.538985
min,0.000000,0.000000,-63.000000,0.000000,-1.158630e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18349.500000,0.000000,0.000000,33.097538,-8.542278e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37014.000000,0.000000,367.000000,34.910989,-5.011919e+00,0.000000,12.000000,3.000000,25.000000,1985.000000
75%,55811.000000,20.000000,1317.250000,37.194164,-3.323050e+00,0.000000,17.000000,5.000000,210.000000,2004.000000
max,74246.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [ ]:
df_train_lab.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [ ]:
#Merge lab and val
df_train = df_train_val.merge(df_train_lab, on='id', how='left')
df_train.shape

(59400, 41)

# Sanity Checks

In [ ]:
df_train.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    2242
gps_height                   0
installer                 2260
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 231
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            2097
recorded_by                  1
scheme_management         2464
scheme_name              17670
permit                    1900
construction_year            1
extraction_type              1
extraction_type_group        1
extraction_type_class        1
management                   1
management_group             1
payment                      1
payment_type                 1
water_quality                1
quality_

In [ ]:
df_train_val['funder'] = ['Other' if i in df_train_val['funder'].value_counts()[8:].index.values else i for i in df_train_val['funder']]
df_train_val['funder'].value_counts()

Other                     21933
Government Of Tanzania     5704
Danida                     1944
Hesawa                     1387
World Bank                  876
Rwssp                       868
Kkkt                        803
World Vision                785
Unicef                      682
Name: funder, dtype: int64

In [ ]:
df_train_val['installer'] = ['Other' if i in df_train_val['installer'].value_counts()[5:].index.values else i for i in df_train_val['installer']]
df_train_val['installer'].value_counts()[:20]

Other         20822
DWE           10918
Government     1151
RWE             765
DANIDA          662
Commu           646
Name: installer, dtype: int64

In [ ]:
df_train_val['wpt_name'] = ['Other' if i in df_train_val['wpt_name'].value_counts()[8:].index.values else i for i in df_train_val['wpt_name']]
df_train_val['wpt_name'].value_counts()[:20]

Other        32422
none          2219
Shuleni       1092
Zahanati       505
Msikitini      302
Kanisani       198
Sokoni         168
Ofisini        161
Bombani        157
Name: wpt_name, dtype: int64

In [ ]:
df_train_val['basin'].value_counts()[:20]

Lake Victoria              6410
Pangani                    5629
Rufiji                     4952
Internal                   4909
Lake Tanganyika            4030
Wami / Ruvu                3742
Lake Nyasa                 3166
Ruvuma / Southern Coast    2853
Lake Rukwa                 1533
Name: basin, dtype: int64

In [ ]:
df_train_val['subvillage'] = ['Other' if i in df_train_val['subvillage'].value_counts()[6:].index.values else i for i in df_train_val['subvillage']]
df_train_val['subvillage'].value_counts()[:20]

Other       35487
Madukani      325
Majengo       319
Shuleni       312
Kati          239
Mtakuja       165
Sokoni        146
Name: subvillage, dtype: int64

In [ ]:
df_train_val['region'].value_counts()[:20]

Iringa         3312
Shinyanga      3147
Mbeya          2900
Kilimanjaro    2773
Morogoro       2486
Kagera         2111
Arusha         2096
Mwanza         1955
Kigoma         1718
Pwani          1634
Ruvuma         1632
Tanga          1614
Dodoma         1379
Singida        1345
Mara           1214
Tabora         1212
Rukwa          1110
Mtwara         1103
Lindi          1000
Manyara         976
Name: region, dtype: int64

In [ ]:
df_train_val['lga'].value_counts()[:20]

Njombe           1579
Moshi Rural       786
Arusha Rural      775
Bariadi           730
Kilosa            687
Rungwe            666
Meru              648
Kasulu            647
Mbozi             639
Bagamoyo          631
Singida Rural     623
Kilombero         589
Same              566
Kahama            556
Kyela             550
Kibondo           523
Karagwe           513
Kigoma Rural      501
Magu              500
Maswa             484
Name: lga, dtype: int64

In [ ]:
df_train_val['ward'].value_counts()[:20]

Igosi           195
Siha Kati       145
Nduruma         142
Imalinyi        141
Mdandu          140
Kitunda         135
Usuka           125
Maji ya Chai    121
Msindo          121
Chalinze        121
Mishamo         118
Chanika         113
Ngarenanyuki    112
Mtwango          99
Matola           98
Vikindu          97
Wanging'ombe     90
Itete            89
Olkokola         88
Igongolo         87
Name: ward, dtype: int64

In [ ]:
df_train_val['recorded_by'].value_counts()[:20]

GeoData Consultants Ltd    37223
Name: recorded_by, dtype: int64

In [ ]:
df_train_val['scheme_management'].value_counts()[:20]

VWC                 23054
WUG                  3241
Water authority      1972
WUA                  1797
Water Board          1736
Parastatal           1051
Company               675
Private operator      662
Other                 462
SWC                    61
Trust                  48
None                    1
Name: scheme_management, dtype: int64

In [ ]:
df_train_val['scheme_name'].value_counts()[:10]

K                              414
None                           407
Borehole                       355
Chalinze wate                  257
DANIDA                         233
M                              230
Government                     204
Ngana water supplied scheme    175
wanging'ombe water supply s    174
wanging'ombe supply scheme     149
Name: scheme_name, dtype: int64

In [ ]:
df_train_lab['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

# Data Type Fixes

In [ ]:
df_test_copy = df_test.copy()

In [ ]:
df_train['date_recorded'] = pd.to_datetime(df_train['date_recorded'])
df_test['date_recorded'] = pd.to_datetime(df_test['date_recorded'])

In [ ]:
df_train['date_recorded_year'] = df_train['date_recorded'].dt.year
df_train['date_recorded_month'] = df_train['date_recorded'].dt.month
df_train['date_recorded_day'] = df_train['date_recorded'].dt.day
df_train['date_recorded_dayweek'] = df_train['date_recorded'].dt.dayofweek
df_train['date_recorded_quarter'] = df_train['date_recorded'].dt.quarter

In [ ]:
df_test['date_recorded_year'] = df_test['date_recorded'].dt.year
df_test['date_recorded_month'] = df_test['date_recorded'].dt.month
df_test['date_recorded_day'] = df_test['date_recorded'].dt.day
df_test['date_recorded_dayweek'] = df_test['date_recorded'].dt.dayofweek
df_test['date_recorded_quarter'] = df_test['date_recorded'].dt.quarter

In [ ]:
#df_train_val_encoded.columns[:50]

In [ ]:
df_train.drop('date_recorded',axis=1,inplace=True)
df_test.drop('date_recorded',axis=1,inplace=True)

In [ ]:
df_train.drop('id',axis=1,inplace=True)
df_test.drop('id',axis=1,inplace=True)

# CatBoost

## CatBoost Data Prep

In [ ]:
cat_features = df_train.drop('status_group',axis=1).select_dtypes('object').columns.values
cat_features

array(['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region',
       'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source',
       'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'], dtype=object)

In [ ]:
df_train[cat_features] = df_train[cat_features].astype(str)
df_test[cat_features] = df_test[cat_features].astype(str)

In [ ]:
# Unnecessary given form data received in but anyway
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('status_group',axis=1), 
                                                    df_train['status_group'], 
                                                    test_size=0.30, random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
Random_oversampler = RandomOverSampler(random_state=42)
X_train, y_train = Random_oversampler.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

non functional             22535
functional                 22535
functional needs repair    22535
Name: status_group, dtype: int64

## CatBoost Feature Selection

In [ ]:
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=200,
                           learning_rate=.03,
                           depth=2,
                           random_state=42,
                           loss_function='MultiClass',
                           eval_metric='AUC',
                           cat_features=cat_features)

In [ ]:
selected_features = model.select_features(X_train,
                         y_train, 
                         [(X_test, y_test)],
                         features_for_select=list(range(X_test.shape[1])),
                         steps=5,
                         num_features_to_select=13,
                         shap_calc_type='Exact',
                         logging_level='Verbose',
                         #train_final_model=False,
                         plot='True')

selected_features = dict(selected_features)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Step #1 out of 5
0:	test: 0.5968862	best: 0.5968862 (0)	total: 397ms	remaining: 1m 18s
1:	test: 0.6435567	best: 0.6435567 (1)	total: 732ms	remaining: 1m 12s
2:	test: 0.6414613	best: 0.6435567 (1)	total: 980ms	remaining: 1m 4s
3:	test: 0.6412856	best: 0.6435567 (1)	total: 1.27s	remaining: 1m 2s
4:	test: 0.6416087	best: 0.6435567 (1)	total: 1.44s	remaining: 56.1s
5:	test: 0.6490114	best: 0.6490114 (5)	total: 1.66s	remaining: 53.5s
6:	test: 0.6463724	best: 0.6490114 (5)	total: 1.82s	remaining: 50.2s
7:	test: 0.6504931	best: 0.6504931 (7)	total: 2.06s	remaining: 49.4s
8:	test: 0.6505764	best: 0.6505764 (8)	total: 2.22s	remaining: 47.1s
9:	test: 0.6509242	best: 0.6509242 (9)	total: 2.4s	remaining: 45.6s
10:	test: 0.6502177	best: 0.6509242 (9)	total: 2.56s	remaining: 44.1s
11:	test: 0.6477673	best: 0.6509242 (9)	total: 2.73s	remaining: 42.7s
12:	test: 0.6395988	best: 0.6509242 (9)	total: 2.91s	remaining: 41.8s
13:	test: 0.6385055	best: 0.6509242 (9)	total: 3.07s	remaining: 40.8s
14:	test: 0.

In [ ]:
model.get_best_score()

{'learn': {'MultiClass': 0.43798614787189494},
 'validation': {'AUC:type=Mu': 0.8347407029305532,
  'MultiClass': 0.6565112612041677}}

In [ ]:
selected_features_names = selected_features['selected_features_names']
selected_features['selected_features_names']

['funder',
 'installer',
 'wpt_name',
 'subvillage',
 'lga',
 'ward',
 'scheme_name',
 'extraction_type',
 'extraction_type_group',
 'quantity',
 'quantity_group',
 'waterpoint_type',
 'waterpoint_type_group']

In [ ]:
selected_features_categFs = []
for selected_feat in selected_features_names:
    if selected_feat in cat_features:
        selected_features_categFs.append(selected_feat)

## CatBoost Model

In [ ]:
# Fit model
CatB_clf = CatBoostClassifier(random_state=42)
CatB_clf.fit(X_train[selected_features_names], y_train, cat_features=selected_features_categFs)

TypeError: ignored

In [ ]:
# Built in feature importances
ctb_feat_importances = pd.DataFrame(CatB_clf.feature_importances_,
                                       CatB_clf.feature_names_,
                                       columns=['feature importance']).sort_values('feature importance',
                                                                                   ascending=False)
ctb_feat_importances

In [ ]:
X_test[cat_features[:15]]

In [ ]:
# Get predicted classes
preds_class = CatB_clf.predict(df_test[selected_features_names])
# Get predicted probabilities for each class
preds_proba = CatB_clf.predict_proba(df_test[selected_features_names])[:,1]
pd.DataFrame(pd.DataFrame(preds_class),df_test_copy['id'])

#XGBoost

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(dataframe, columns):
    """
    Label encodes the specified columns of a dataframe.
    """
    for column in columns:
      le = LabelEncoder()
      dataframe[column] = le.fit_transform(dataframe[column])
      print(le.classes_)
    return dataframe
df_train_encoded = label_encode_columns(df_train, df_train.select_dtypes('object').columns.values)
df_test_encoded = label_encode_columns(df_test, df_test.select_dtypes('object').columns.values)

In [ ]:
# Unnecessary given the form the data was received in but anyway
X_train, X_test, y_train, y_test = train_test_split(df_train_encoded.drop('status_group',axis=1), 
                                                    df_train_encoded['status_group'], 
                                                    test_size=0.30, random_state=42)

In [ ]:
Random_oversampler = RandomOverSampler(random_state=42)
X_train, y_train = Random_oversampler.fit_resample(X_train, y_train)

In [ ]:
# Step 2: Create pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(seed = 42))
])

# Step 3: Fit the pipeline to the training data
pipe.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = pipe.predict(X_test)
y_probs = pipe.predict_proba(X_test)

# Step 5: Evaluate the model performance
print(classification_report(y_test, pd.Series(y_pred)))
f1, precision, recal, roc_auc = eval_metrics(y_test, pd.Series(y_pred), y_probs, ave='macro')
print(precision, recal, roc_auc)

              precision    recall  f1-score   support

           0       0.71      0.92      0.80      9724
           1       0.58      0.10      0.17      1293
           2       0.83      0.61      0.70      6803

    accuracy                           0.74     17820
   macro avg       0.71      0.54      0.56     17820
weighted avg       0.75      0.74      0.72     17820

0.706 0.542 0.822


In [ ]:
cm = metrics.multilabel_confusion_matrix(y_test, y_pred)#.confusion_matrix(y_val, predictions)
precision=[]
sensitivity=[]
specificity=[]
npv=[]
accuracy=[]
for i,obj in enumerate(cm):
  (tn, fp), (fn, tp) = cm[i]#.ravel()
  # Precision = TP / (TP + FP)
  precision.append(tp / (tp + fp))
  # Sensitivity = TP / (TP + FN)
  sensitivity.append(tp / (tp + fn))
  # Specificity = TN / (TN + FP)
  specificity.append(tn / (tn + fp))
  # NPV = TN / (TN + FN)
  npv.append(tn / (tn + fn))
  # Accuracy = (TP + TN) / (TP + TN + FP + FN)
  accuracy.append((tp + tn) / (tp + tn + fp + fn))
pd.DataFrame([precision,sensitivity,specificity,npv,accuracy]).transpose()

,0,1,2,3,4
0,0.711321,0.923385,0.549901,0.856648,0.753704
1,0.580645,0.097448,0.994494,0.933704,0.929405
2,0.827309,0.605615,0.921939,0.791044,0.801178


In [ ]:
preds_class = pipe.predict(df_test)
pd.DataFrame(preds_class,df_test_copy['id'])

,0
id,
50785,2
51630,0
17168,2
45559,2
49871,0
...,...
39307,2
18990,0
28749,0


## GridSearchCV

In [ ]:
# Step 1: Define the parameters for the xgboost classifier
param_grid = {
    'classifier__max_depth': range(2,10,4),
    'classifier__n_estimators': [50, 100],
    'classifier__learning_rate': [0.2, 0.1, 0.05]
}

# Step 3: Create pipeline
pipe_grid = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(seed = 42))
])

# Step 4: Hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(pipe_grid, param_grid=param_grid, cv=5, scoring='recall', n_jobs=-1, verbose=True)
grid_search.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = grid_search.predict(X_test)

# Step 6: Evaluate the model performance
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:969: UserWarning:

One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]



              precision    recall  f1-score   support

           0       0.69      0.94      0.80      9724
           1       0.69      0.05      0.09      1293
           2       0.84      0.56      0.67      6803

    accuracy                           0.73     17820
   macro avg       0.74      0.51      0.52     17820
weighted avg       0.75      0.73      0.70     17820



In [ ]:
preds_class = grid_search.predict(df_test)
pd.DataFrame(preds_class,df_test_copy['id'])

,0
id,
50785,2
51630,0
17168,2
45559,2
49871,0
...,...
39307,2
18990,0
28749,0


In [ ]:
grid_search.best_params_

{'classifier__learning_rate': 0.1,
 'classifier__max_depth': 2,
 'classifier__n_estimators': 100}

In [ ]:
cm = metrics.multilabel_confusion_matrix(y_test, y_pred)#.confusion_matrix(y_val, predictions)
precision=[]
sensitivity=[]
specificity=[]
npv=[]
accuracy=[]
for i,obj in enumerate(cm):
  (tn, fp), (fn, tp) = cm[i]#.ravel()
  # Precision = TP / (TP + FP)
  precision.append(tp / (tp + fp))
  # Sensitivity = TP / (TP + FN)
  sensitivity.append(tp / (tp + fn))
  # Specificity = TN / (TN + FP)
  specificity.append(tn / (tn + fp))
  # NPV = TN / (TN + FN)
  npv.append(tn / (tn + fn))
  # Accuracy = (TP + TN) / (TP + TN + FP + FN)
  accuracy.append((tp + tn) / (tp + tn + fp + fn))
pd.DataFrame([precision,sensitivity,specificity,npv,accuracy]).transpose()

# Random Forest

In [ ]:
# Step 2: Create pipeline
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_jobs=-1,random_state=42, verbose=True))
])

# Step 3: Fit the pipeline to the training data
pipe_rf.fit(X_train, y_train)

# Step 4: Make predictions on the test set
y_pred = pipe_rf.predict(X_test)
y_probs = pipe_rf.predict_proba(X_test)

# Step 5: Evaluate the model performance
print(classification_report(y_test, y_pred))
f1, precision, recal, roc_auc = eval_metrics(y_test, pd.Series(y_pred), y_probs, ave='micro')
print(precision, recal, roc_auc)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   12.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s


              precision    recall  f1-score   support

           0       0.82      0.86      0.84      9724
           1       0.46      0.44      0.45      1293
           2       0.83      0.79      0.81      6803

    accuracy                           0.80     17820
   macro avg       0.71      0.69      0.70     17820
weighted avg       0.80      0.80      0.80     17820

0.801 0.801 0.887


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.4s finished


In [ ]:
cm = metrics.multilabel_confusion_matrix(y_test, y_pred)#.confusion_matrix(y_val, predictions)
precision=[]
sensitivity=[]
specificity=[]
npv=[]
accuracy=[]
for i,obj in enumerate(cm):
  (tn, fp), (fn, tp) = cm[i]#.ravel()
  # Precision = TP / (TP + FP)
  precision.append(tp / (tp + fp))
  # Sensitivity = TP / (TP + FN)
  sensitivity.append(tp / (tp + fn))
  # Specificity = TN / (TN + FP)
  specificity.append(tn / (tn + fp))
  # NPV = TN / (TN + FN)
  npv.append(tn / (tn + fn))
  # Accuracy = (TP + TN) / (TP + TN + FP + FN)
  accuracy.append((tp + tn) / (tp + tn + fp + fn))
pd.DataFrame([precision,sensitivity,specificity,npv,accuracy]).transpose()

,0,1,2,3,4
0,0.821583,0.860448,0.775568,0.822289,0.821886
1,0.460720,0.435422,0.960126,0.956019,0.922054
2,0.833022,0.785389,0.902787,0.871997,0.857969


In [ ]:
preds_class = pipe_rf.predict(df_test)
pd.DataFrame(preds_class,df_test_copy['id'])

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.4s finished


,0
id,
50785,2
51630,0
17168,2
45559,2
49871,0
...,...
39307,2
18990,0
28749,0


## GridSearchCV

In [ ]:
# Step 1: Define the parameters for the xgboost classifier
param_grid = {
    'classifier__max_depth': [2,5,10,15],
    'classifier__max_features': ['sqrt','log2'],
    #'classifier__learning_rate': [0.1, 0.01, 0.001]
}

# Step 3: Create pipeline
pipe_grid = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_jobs=-1,random_state=42, verbose=True))
])

# Step 4: Hyperparameter tuning using GridSearchCV
grid_search_rf = GridSearchCV(pipe_grid, param_grid=param_grid, cv=5, scoring='precision', n_jobs=-1, verbose=True)
grid_search_rf.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = grid_search_rf.predict(X_test)

# Step 6: Evaluate the model performance
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:969: UserWarning:

One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan]

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.0s


              precision    recall  f1-score   support

           0       0.70      0.56      0.63      9724
           1       0.14      0.55      0.23      1293
           2       0.72      0.53      0.61      6803

    accuracy                           0.55     17820
   macro avg       0.52      0.55      0.49     17820
weighted avg       0.67      0.55      0.59     17820



[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


In [ ]:
preds_class = grid_search_rf.predict(df_test)
pd.DataFrame(preds_class,df_test_copy['id'])

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


,0
id,
50785,2
51630,0
17168,2
45559,2
49871,0
...,...
39307,0
18990,0
28749,0


In [ ]:
grid_search_rf.best_params_

{'classifier__max_depth': 2, 'classifier__max_features': 'sqrt'}

In [ ]:
cm = metrics.multilabel_confusion_matrix(y_test, y_pred)#.confusion_matrix(y_val, predictions)
precision=[]
sensitivity=[]
specificity=[]
npv=[]
accuracy=[]
for i,obj in enumerate(cm):
  (tn, fp), (fn, tp) = cm[i]#.ravel()
  # Precision = TP / (TP + FP)
  precision.append(tp / (tp + fp))
  # Sensitivity = TP / (TP + FN)
  sensitivity.append(tp / (tp + fn))
  # Specificity = TN / (TN + FP)
  specificity.append(tn / (tn + fp))
  # NPV = TN / (TN + FN)
  npv.append(tn / (tn + fn))
  # Accuracy = (TP + TN) / (TP + TN + FP + FN)
  accuracy.append((tp + tn) / (tp + tn + fp + fn))
pd.DataFrame([precision,sensitivity,specificity,npv,accuracy]).transpose()

,0,1,2,3,4
0,0.702373,0.563040,0.713439,0.576160,0.631369
1,0.142658,0.553751,0.739638,0.954925,0.726150
2,0.721734,0.531089,0.873559,0.751054,0.742817
